In [1]:
import torch
from torch.utils import data
import torchvision
from torch import nn
from torch.utils.tensorboard import SummaryWriter

trans = torchvision.transforms.ToTensor()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mnist_train = torchvision.datasets.FashionMNIST(root = 'E:\Deep_learning\data' , train = True , transform = trans , download = True)
mnist_test = torchvision.datasets.FashionMNIST(root = 'E:\Deep_learning\data' , train = False , transform = trans , download = True)

batch_size = 256

train_iter = data.DataLoader(mnist_train , batch_size , shuffle = True , num_workers = 0 , drop_last = False)
text_iter = data.DataLoader(mnist_test , batch_size , shuffle = False , num_workers = 0 , drop_last = False)

In [2]:
import torch.nn.functional as F

inputs_num = 784
hidden_num = 256
outputs_num = 10

class Net(nn.Module):
    def __init__(self,inputs_num , hidden_num , outputs_num ,*args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.fl = nn.Flatten()
        self.fc1 = nn.Linear(inputs_num ,  hidden_num)
        self.fc2 = nn.Linear(hidden_num , outputs_num)
    
    def forward(self , x):
        x = self.fl(x)
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x))
        return x

net = Net(inputs_num , hidden_num , outputs_num).to(device)
print(net)

Net(
  (fl): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)


In [3]:
loss = nn.CrossEntropyLoss()
loss = loss.to(device)
trainer = torch.optim.SGD(net.parameters(), lr=0.01)
num_epochs = 1000
total_train_step = 0
total_test_step = 0

writer = SummaryWriter("log")

for epoch in range(num_epochs):
    #训练
    print("----------第{}轮训练开始----------".format(epoch + 1))
    net.train()
    for data in train_iter:
        X , y = data
        X = X.to(device)
        y = y.to(device)
        y_hat = net(X)
        l = loss(y_hat , y)
        trainer.zero_grad()
        l.mean().backward()
        trainer.step()
        total_train_step += 1
        if total_train_step % 100 == 0:
            #print("训练次数{},Loss:{}".format(total_train_step,l.item()))
            writer.add_scalar("train_loss",l.item(),total_train_step)
    #测试
    net.eval()
    total_test_loss = 0
    total_accurancy = 0
    with torch.no_grad():
        for data in  text_iter:
            X , y = data
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat , y)
            total_test_loss += l.item()
            accuracy = (y_hat.argmax(1) == y).sum()
            total_accurancy += accuracy
    #print("整体测试集上的Loss：{}".format(total_test_loss))
    #print("整体测试集上的正确率：{}".format(total_accurancy / len(mnist_test)))
    writer.add_scalar("test_loss",total_test_loss,total_test_step)
    writer.add_scalar("test_accurancy",total_accurancy / len(mnist_test),total_test_step)
    total_test_step += 1
    
    torch.save(net , "E:/Deep_learning/m/2net_{}.pth".format(epoch + 1))
    print("模型已保存")

writer.close()

----------第1轮训练开始----------


C:\Users\Jiangwang\AppData\Local\Temp\ipykernel_18464\1469341930.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.fc2(x))


模型已保存
----------第2轮训练开始----------
模型已保存
----------第3轮训练开始----------
模型已保存
----------第4轮训练开始----------
模型已保存
----------第5轮训练开始----------
模型已保存
----------第6轮训练开始----------
模型已保存
----------第7轮训练开始----------
模型已保存
----------第8轮训练开始----------
模型已保存
----------第9轮训练开始----------
模型已保存
----------第10轮训练开始----------
模型已保存
----------第11轮训练开始----------
模型已保存
----------第12轮训练开始----------
模型已保存
----------第13轮训练开始----------
模型已保存
----------第14轮训练开始----------
模型已保存
----------第15轮训练开始----------
模型已保存
----------第16轮训练开始----------
模型已保存
----------第17轮训练开始----------
模型已保存
----------第18轮训练开始----------
模型已保存
----------第19轮训练开始----------
模型已保存
----------第20轮训练开始----------
模型已保存
----------第21轮训练开始----------
模型已保存
----------第22轮训练开始----------
模型已保存
----------第23轮训练开始----------
模型已保存
----------第24轮训练开始----------
模型已保存
----------第25轮训练开始----------
模型已保存
----------第26轮训练开始----------
模型已保存
----------第27轮训练开始----------
模型已保存
----------第28轮训练开始----------
模型已保存
----------第29轮训练开始----------
模型已保存
----------第30轮训练开始----

KeyboardInterrupt: 